### MS&E 346 Assignment 6
#### January 30

In [3]:
import numpy as np
from scipy.linalg import eig
import copy
import math
from typing import Dict, Tuple, Sequence

#### Question 1:
Implement Black-Scholes formulas for European Call/Put Pricing. Implement standard binary tree/grid-based numerical algorithm for American Option Pricing and ensure it validates against Black-Scholes formula for Europeans
#### Answer:

the underlying asset follows a geometric brownian motion
${\frac{dS_t}{s_t} = \mu dt + \sigma dW_t}$
The Black-Scholes model:  
value of a call: ${SN(d_1) - Ke^{-rt}N(d_2)}$
where ${d_1 = \frac{ln(\frac{S}{K}) + (r + \frac{\sigma^2}{2})t}{\sigma \sqrt{t}}}$  
${d_2 = d_1 - \sigma \sqrt{t}}$

In [4]:
class bsmodel:
    """
    The value of a call option in Black-Scholes model can be written as a function of the following variables
    S = current value of the underlying asset
    K = strike price of the option
    t = Life to expiration of the option
    r = risk-free rate
    sigma = standard deviation in ln value of the underlying asset
    """
    def get_d1_d2(self) -> Tuple:
        d1 = (np.log(self.s / self.k) +
              (self.r + self.sigma ** 2 / 2.) * self.t) / (self.sigma * np.sqrt(self.t))
        d2 = d1 - self.sigma * np.sqrt(self.t)
        self.d1 = d1
        self.d2 = d2
        return d1, d2

    def get_option_price(self) -> float:
        d1, d2 = self.get_d1_d2()
        if self.call:
            c = self.s * norm.cdf(d1) - self.k * np.exp(-self.r * self.t) * norm.cdf(d2)
        else:
            c = self.k * np.exp(-self.r * self.t) * norm.cdf(-d2) - self.spot_price * norm.cdf(-d1)
        print (10*"*", 'The value of the underlying option is %f.' % c) 
        return c
    
    def __init__(self, s, k, r, t, mu, sigma, call=False):
        self.s = s
        self.k = k
        self.r = r
        self.t = t
        self.mu = mu
        self.sigma = sigma
        self.call = call
        _, _ = self.get_d1_d2()
        self.c = get_option_price()
        

#### Question 2:
Implement Longstaff-Schwartz Algorithm and ensure it validates against binary tree/grid-based solution for path-independent options. 
#### Answer:

* m Monte-Carlo paths indexed i = 0, 1, ..., m - 1
* n + 1 time steps indexed j = n, n - 1, ..., 1, 0
* Infitesimal Risk-free rate at time ${t_j}$ denoted ${r_{t_j}}$
* At each time step, ${CF[i]}$ is the present value and future cashflow of path i
* ${s_{i,j}}$ denotes state for ${(i,j) := (time t_j, price history SP[i, :(j+1)])}$
* Regression function ${f(s_{i,j}) = w \phi(s_{i,j}) = \Sigma_{l=0}^{r-1}w_l\phi_l(S_{i,j})}$

In [8]:
# there are m monte-carlo paths and n + 1 time step
import sklearn.metrics.pairwise # using rbf_kernel here to list the features of a state
from sklearn.linear_model import LinearRegression
from numpy.linalg import inv

class longstaffschwartz:
    def init_weight(self, n):
        # initialize the weight to project a hand-crafted kernel to size of n
        self.weight = np.random.rand((n, 2 * n))
    
    def kernel(self, x):
        # a self-defined kernel to capture the correlation of the state
        length = len(x)
        phi_x = np.zeros((2*length, 1))
        phi_x[lenght:] = np.array(x)^2
        self.kernel_size = phi_x.shape[0]
        
        return phi_x
    
    def ls_iterate(self):
        for j in range(self.n - 1, 0, -1):
            current_risk_free_rate = rate[j]
            payoff = copy.deepcopy(cf)
            cf[0: self.m] = cf[0: self.m] * np.exp(-current_risk_free_rate)
            for i in range(m):
                if self.X is None:
                    self.X = []
                if self.Y is None:
                    self.Y = []
                if cf[i] > 0:
                    X.append(self.kernel(s[i,j]))
                    Y.append(self.cf[i])
                X = np.asarray(X)
                Y = np.asarray(Y)
                # the solution of the LSE estimation 
            self.weight = np.matmul(np.matmul(inv(np.matmul(X.T, X)), X.T),Y)
            for i in range(m):
                if payoff[i] > np.dot(self.weight, self.kernel(s[i,:])):
                    cf[i] = payoff[i] 
        exer = cf[0]
        cont = np.exp(-self.rate[0]) * np.mean(cf)
        return np.max(exer, cont)
            
    
    def __init__(self, cf, m, n, rate, s):
        self.m = m
        self.n = n
        self.rate = rate
        self.X = None
        self.Y = None
        self.s = s
        # At maturity, the value of an option is hm
        if cf is None:
            cf = np.random.rand((1,n))
        self.cf = cf
        # construct the estimation of the continuation value
        self.c_m = np.zeros((n + 1, m))
        self.c_m_hat = np.zeros((n + 1, m))
        self.c_m[n,:] = hm
        # initialize the weight matrix
        self.init_weight(n)
        
        
        